In [11]:
import json

from utils import *
from Deep_Learning_Tools import *
from Data_Tools import *

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import SGD
from keras.regularizers import l1

import logging
my_seed=1
set_libraries_seeds(my_seed) #Important to approach reproductible results 
#tf.logging.set_verbosity(tf.logging.ERROR) #setting a lower verbosity
logging.getLogger("tensorflow").setLevel(logging.ERROR)

# MQ2008

In [2]:
epochs=[100,500,1000]
learning_rates=[0.001,0.075]
mon=[0.01,0.1,0.3]
out_act=['Identity','Sigmoid']
hid_act= ['Identity','Sigmoid','ReLu']
number_hidden_layers = [0,1]
number_neurons = [15,46]


In [3]:
len(epochs)*len(learning_rates)*len(mon)*len(out_act)*len(hid_act)\
    *(len(number_hidden_layers)-1)*len(number_neurons)

216

In [4]:
if 0 in number_hidden_layers:
    total_it = len(epochs)*len(learning_rates)*len(mon)*len(out_act)
    total_it += len(epochs)*len(learning_rates)*len(mon)*len(out_act)*len(hid_act)\
    *(len(number_hidden_layers)-1)*len(number_neurons)
else:
    total_it = len(epochs)*len(learning_rates)*len(mon)*len(out_act)*len(hid_act)\
    *len(number_hidden_layers)*len(number_neurons)

In [5]:
total_it

252

In [6]:
### As ltr4l automatically compute the NDCG score without giving the final ranking
### we instead use the model created and use it on our datasets with tensorflow

act_func_mapper = {
    "Sigmoid" : "sigmoid",
    "ReLu" : "relu",
    "Identity" : "linear"
}


for fold in [1,2,3,4,5]:
    ###GET THE DATASETS
    path='../../MQ2008/Fold'+str(fold)+'/'

    data_vali=load_data(path+'vali.txt')
    data_train=load_data(path+'train.txt')
    data_test=load_data(path+'test.txt')
    
    model_counter = 1

    for my_iter in range(total_it):
        ###CONSTRUCT THE MODEL ON TENSORFLOW BASED ON THE JSON CONSTRUCTED BY LTR4L
        name = "ltr4l"+str(model_counter)
        
        model_path = "../../MQ2008/Fold"+str(fold)+"/models/ltr4l_"+str(model_counter)+".json"
        with open(model_path, 'r') as f:
            model = json.load(f)
        
        nb_layers = len(model["config"]["params"]["layers"])+1
        layer_configs = model["config"]["params"]["layers"]
        
        x_list_train,y_list_train=get_list_xy(data_train)
        modelq = Sequential()
        
        act = "linear"
        nb_neurons_out = layer_configs[0]["num"]
        nb_neurons_in = x_list_train[0].shape[1]
        
        modelq.add(Dense(nb_neurons_out,input_dim=nb_neurons_in, activation = act))
        
        ###SETTING THE WEIGTH OF EACH NEURON OF THE EXTERN LAYERS
        bias = np.array([item[0] for item in model["weights"][0]])
        bias = bias.reshape(nb_neurons_out)
        weights = np.array([item[1:] for item in model["weights"][0]])
        weights = weights.reshape(nb_neurons_in,nb_neurons_out)

        modelq.layers[0].set_weights((weights,bias))
        
        #opt = SGD(lr=n_, momentum=mom_)

        #modelq.compile(loss=Loss_query_keras, optimizer=opt)
        
        ###SETTING THE HIDDEN LAYERS
        for layer in range(1,nb_layers):
            act = act_func_mapper[layer_configs[layer-1]["activator"]]
            nb_neurons_out = 1
            nb_neurons_in = layer_configs[layer-1]["num"]
            if layer < nb_layers-1:
                nb_neurons_out = layer_configs[layer]["num"]
            
            modelq.add(Dense(nb_neurons_out,input_dim = nb_neurons_in, activation = act))
            
            ###SETTING THE WEIGTH OF EACH NEURON OF THE HIDDEN LAYERS
            bias = np.array([item[0] for item in model["weights"][layer]])
            bias = bias.reshape(nb_neurons_out)
            weights = np.array([item[1:] for item in model["weights"][layer]])
            weights = weights.reshape(nb_neurons_in,nb_neurons_out)
            
            modelq.layers[layer].set_weights((weights,bias))

            
        ###USE THE MODELS TO PREDICT THE RANKING OF EACH SET
        y_pred_train=modelq.predict(data_train.drop(['relevance degree','qid'],axis=1)).astype('float64').ravel()
        df_train=pd.DataFrame(y_pred_train)
        df_train.to_csv(path+'2020_10_02_predictions/y_train_'+name+'.txt',sep=' ',header=False,index=False)

        y_pred_vali=modelq.predict(data_vali.drop(['relevance degree','qid'],axis=1)).astype('float64').ravel()
        df_vali=pd.DataFrame(y_pred_vali)
        df_vali.to_csv(path+'2020_10_02_predictions/y_vali_'+name+'.txt',sep=' ',header=False,index=False)

        y_pred_test=modelq.predict(data_test.drop(['relevance degree','qid'],axis=1)).astype('float64').ravel()
        df_test=pd.DataFrame(y_pred_test)
        df_test.to_csv(path+'2020_10_02_predictions/y_test_'+name+'.txt',sep=' ',header=False,index=False)
        
        model_counter += 1

# MQ2007

In [12]:
epochs=[100,500]
learning_rates=[0.001,0.075]
mon=[0.01,0.1,0.3]
out_act=['Identity','Sigmoid']
hid_act= ['Identity','Sigmoid','ReLu']
number_hidden_layers = [0,1]
number_neurons = [15,46]

In [13]:
if 0 in number_hidden_layers:
    total_it = len(epochs)*len(learning_rates)*len(mon)*len(out_act)
    total_it += len(epochs)*len(learning_rates)*len(mon)*len(out_act)*len(hid_act)\
    *(len(number_hidden_layers)-1)*len(number_neurons)
else:
    total_it = len(epochs)*len(learning_rates)*len(mon)*len(out_act)*len(hid_act)\
    *len(number_hidden_layers)*len(number_neurons)

In [15]:
### As ltr4l automatically compute the NDCG score without giving the final ranking
### we instead use the model created and use it on our datasets with tensorflow


act_func_mapper = {
    "Sigmoid" : "sigmoid",
    "ReLu" : "relu",
    "Identity" : "linear"
}


for fold in [1,2,3,4,5]:
    ###CONSTRUCT THE MODEL ON TENSORFLOW BASED ON THE JSON CONSTRUCTED BY LTR4L
    
    path='../../MQ2007/Fold'+str(fold)+'/'

    data_vali=load_data(path+'vali.txt')
    data_train=load_data(path+'train.txt')
    data_test=load_data(path+'test.txt')
    
    model_counter = 1

    for my_iter in range(total_it):
        
        name = "ltr4l"+str(model_counter)
        
        model_path = "../../MQ2007/Fold"+str(fold)+"/models/ltr4l_"+str(model_counter)+".json"
        with open(model_path, 'r') as f:
            model = json.load(f)
        
        nb_layers = len(model["config"]["params"]["layers"])+1
        layer_configs = model["config"]["params"]["layers"]
        
        x_list_train,y_list_train=get_list_xy(data_train)
        modelq = Sequential()
        
        act = "linear"
        nb_neurons_out = layer_configs[0]["num"]
        nb_neurons_in = x_list_train[0].shape[1]
        
        modelq.add(Dense(nb_neurons_out,input_dim=nb_neurons_in, activation = act))
        
        ###SETTING THE WEIGTH OF EACH NEURON OF THE EXTERN LAYERS
        bias = np.array([item[0] for item in model["weights"][0]])
        bias = bias.reshape(nb_neurons_out)
        weights = np.array([item[1:] for item in model["weights"][0]])
        weights = weights.reshape(nb_neurons_in,nb_neurons_out)

        modelq.layers[0].set_weights((weights,bias))
        
        #opt = SGD(lr=n_, momentum=mom_)

        #modelq.compile(loss=Loss_query_keras, optimizer=opt)
        
        ###SETTING THE HIDDEN LAYERS
        for layer in range(1,nb_layers):
            act = act_func_mapper[layer_configs[layer-1]["activator"]]
            nb_neurons_out = 1
            nb_neurons_in = layer_configs[layer-1]["num"]
            if layer < nb_layers-1:
                nb_neurons_out = layer_configs[layer]["num"]
            
            modelq.add(Dense(nb_neurons_out,input_dim = nb_neurons_in, activation = act))
            
            ###SETTING THE WEIGTH OF EACH NEURON OF THE HIDDEN LAYERS
            bias = np.array([item[0] for item in model["weights"][layer]])
            bias = bias.reshape(nb_neurons_out)
            weights = np.array([item[1:] for item in model["weights"][layer]])
            weights = weights.reshape(nb_neurons_in,nb_neurons_out)
            
            modelq.layers[layer].set_weights((weights,bias))

        ###USE THE MODELS TO PREDICT THE RANKING OF EACH SET
        y_pred_train=modelq.predict(data_train.drop(['relevance degree','qid'],axis=1)).astype('float64').ravel()
        df_train=pd.DataFrame(y_pred_train)
        df_train.to_csv(path+'new_predictions/y_train_'+name+'.txt',sep=' ',header=False,index=False)

        y_pred_vali=modelq.predict(data_vali.drop(['relevance degree','qid'],axis=1)).astype('float64').ravel()
        df_vali=pd.DataFrame(y_pred_vali)
        df_vali.to_csv(path+'new_predictions/y_vali_'+name+'.txt',sep=' ',header=False,index=False)


        y_pred_test=modelq.predict(data_test.drop(['relevance degree','qid'],axis=1)).astype('float64').ravel()
        df_test=pd.DataFrame(y_pred_test)
        df_test.to_csv(path+'new_predictions/y_test_'+name+'.txt',sep=' ',header=False,index=False)
        
        model_counter += 1